In [104]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import crawler
import Data_preperation
from time import time
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
plt.style.use('ggplot')


In [2]:
url = 'https://www.basketball-reference.com/teams/'
# crawler.makeCoachesDataSet(url)
# Data_preperation.joinDataFile('Data')

In [3]:
df = pd.read_csv('Data/Joined Data.csv')

In [4]:
df = df.sort_values('Year', ascending=False)
df = df.drop(df[df['Year'] == 2020].index)

In [5]:
df['num_of_coaches'] = 0
for indx, row in df.iterrows():
    for i in range(1,5):
        if not pd.isna(row['Coach'+str(i)]):
            df['num_of_coaches'][indx] = df['num_of_coaches'][indx] + 1
df.head()

,Unnamed: 0,Year,Coach1,Balance1,Coach2,Balance2,Coach3,Balance3,Coach4,Balance4,Coach1Wins,Coach1Lose,Coach2Wins,Coach2Lose,Coach3Wins,Coach3Lose,Coach4Wins,Coach4Lose,Total1,Total2,Total3,Total4,Precentage1,Precentage2,Precentage3,Precentage4,num_of_coaches
840,2,2019,Tom Thibodeau,19-21,Ryan Saunders,17-25,NaN,NaN,NaN,NaN,19.0,21.0,17.0,25.0,0.0,0.0,0.0,0.0,40,42,0,0,47,40.0,NaN,NaN,2
172,2,2019,Fred Hoiberg,5-19,Jim Boylen,17-41,NaN,NaN,NaN,NaN,5.0,19.0,17.0,41.0,0.0,0.0,0.0,0.0,24,58,0,0,20,29.0,NaN,NaN,2
226,3,2019,Tyronn Lue,0-6,Larry Drew,19-57,NaN,NaN,NaN,NaN,0.0,6.0,19.0,57.0,0.0,0.0,0.0,0.0,6,76,0,0,0,25.0,NaN,NaN,2
275,1,2019,Rick Carlisle,33-49,NaN,NaN,NaN,NaN,NaN,NaN,33.0,49.0,0.0,0.0,0.0,0.0,0.0,0.0,82,0,0,0,40,NaN,NaN,NaN,1
314,1,2019,Mike Malone,54-28,NaN,NaN,NaN,NaN,NaN,NaN,54.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,82,0,0,0,65,NaN,NaN,NaN,1


In [6]:
df = df.drop(df[df['num_of_coaches'] == 1].index)
df.head()

,Unnamed: 0,Year,Coach1,Balance1,Coach2,Balance2,Coach3,Balance3,Coach4,Balance4,Coach1Wins,Coach1Lose,Coach2Wins,Coach2Lose,Coach3Wins,Coach3Lose,Coach4Wins,Coach4Lose,Total1,Total2,Total3,Total4,Precentage1,Precentage2,Precentage3,Precentage4,num_of_coaches
840,2,2019,Tom Thibodeau,19-21,Ryan Saunders,17-25,NaN,NaN,NaN,NaN,19.0,21.0,17.0,25.0,0.0,0.0,0.0,0.0,40,42,0,0,47,40.0,NaN,NaN,2
172,2,2019,Fred Hoiberg,5-19,Jim Boylen,17-41,NaN,NaN,NaN,NaN,5.0,19.0,17.0,41.0,0.0,0.0,0.0,0.0,24,58,0,0,20,29.0,NaN,NaN,2
226,3,2019,Tyronn Lue,0-6,Larry Drew,19-57,NaN,NaN,NaN,NaN,0.0,6.0,19.0,57.0,0.0,0.0,0.0,0.0,6,76,0,0,0,25.0,NaN,NaN,2
790,3,2018,Jason Kidd,23-22,Joe Prunty,21-16,NaN,NaN,NaN,NaN,23.0,22.0,21.0,16.0,0.0,0.0,0.0,0.0,45,37,0,0,51,56.0,NaN,NaN,2
1167,3,2018,Earl Watson,0-3,Jay Triano,21-58,NaN,NaN,NaN,NaN,0.0,3.0,21.0,58.0,0.0,0.0,0.0,0.0,3,79,0,0,0,26.0,NaN,NaN,2


In [7]:
follow_df = df.copy()
tri_df = df.copy()

In [8]:
follow_df['0_diff_Test1'] = follow_df['0_diff_Test2'] = follow_df['0_diff_Test3'] = False
follow_df['5_diff_Test1'] = follow_df['5_diff_Test2'] = follow_df['5_diff_Test3'] = False

In [14]:
for indx, row in follow_df.iterrows():
    for i in range(1, row['num_of_coaches']):
        follow_df['0_diff_Test'+str(i)][indx] = row['Precentage'+str(i+1)] - row['Precentage'+str(i)] > 0
        follow_df['5_diff_Test'+str(i)][indx] = row['Precentage'+str(i+1)] - row['Precentage'+str(i)] > 5

In [10]:
tri_df['0_diff_Test1-2'] = tri_df['0_diff_Test1-3'] = tri_df['0_diff_Test1-4'] = tri_df['0_diff_Test2-3'] = tri_df['0_diff_Test2-4'] = tri_df['0_diff_Test3-4'] = None
tri_df['5_diff_Test1-2'] = tri_df['5_diff_Test1-3'] = tri_df['5_diff_Test1-4'] = tri_df['5_diff_Test2-3'] = tri_df['5_diff_Test2-4'] = tri_df['5_diff_Test3-4'] = None

In [11]:

for indx, row in follow_df.iterrows():
    for i in range(1, row['num_of_coaches']):
        for j in range(i + 1, row['num_of_coaches'] + 1):
            tri_df['0_diff_Test'+str(i)+'-'+str(j)][indx] = row['Precentage'+str(j)] - row['Precentage'+str(i)] > 0
            tri_df['5_diff_Test'+str(i)+'-'+str(j)][indx] = row['Precentage'+str(j)] - row['Precentage'+str(i)] > 5
tri_df[tri_df['num_of_coaches']>2].head()

,Unnamed: 0,Year,Coach1,Balance1,Coach2,Balance2,Coach3,Balance3,Coach4,Balance4,Coach1Wins,Coach1Lose,Coach2Wins,Coach2Lose,Coach3Wins,Coach3Lose,Coach4Wins,Coach4Lose,Total1,Total2,Total3,Total4,Precentage1,Precentage2,Precentage3,Precentage4,num_of_coaches,0_diff_Test1-2,0_diff_Test1-3,0_diff_Test1-4,0_diff_Test2-3,0_diff_Test2-4,0_diff_Test3-4,5_diff_Test1-2,5_diff_Test1-3,5_diff_Test1-4,5_diff_Test2-3,5_diff_Test2-4,5_diff_Test3-4
1270,7,2015,Mike Malone,11-13,Tyrone Corbin,7-21,George Karl,11-19,NaN,NaN,11.0,13.0,7.0,21.0,11.0,19.0,0.0,0.0,24,28,30,0,45,25.0,36.0,NaN,3,False,False,None,True,None,None,False,False,None,True,None,None
669,9,2013,Mike Brown,1-4,Bernie Bickerstaff,4-1,Mike D'Antoni,40-32,NaN,NaN,1.0,4.0,4.0,1.0,40.0,32.0,0.0,0.0,5,5,72,0,20,80.0,55.0,NaN,3,True,True,None,False,None,None,True,True,None,False,None,None
880,15,2010,Lawrence Frank,0-16,Tom Barrise,0-2,Kiki Vandeweghe,12-52,NaN,NaN,0.0,16.0,0.0,2.0,12.0,52.0,0.0,0.0,16,2,64,0,0,0.0,18.0,NaN,3,False,True,None,True,None,None,False,True,None,True,None,None
744,14,2009,Marc Iavaroni,11-30,Johnny Davis,0-2,Lionel Hollins,13-26,NaN,NaN,11.0,30.0,0.0,2.0,13.0,26.0,0.0,0.0,41,2,39,0,26,0.0,33.0,NaN,3,False,True,None,True,None,None,False,True,None,True,None,None
183,15,2008,Scott Skiles,9-16,Pete Myers,0-1,Jim Boylan,24-32,NaN,NaN,9.0,16.0,0.0,1.0,24.0,32.0,0.0,0.0,25,1,56,0,36,0.0,42.0,NaN,3,False,True,None,True,None,None,False,True,None,True,None,None


In [12]:
################################################
#
# Build estimator function for all tests
#
################################################
def run_methods_estimators(X,y):
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=4)
    scores = []
    scores_knn = []
    max_score = max_k = 0
    k_range = [1,5,11,15,21,31,51]
    for k in k_range:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train,y_train)
        y_pred_knn=knn.predict(X_test)
        score = metrics.accuracy_score(y_test,y_pred_knn)
        if max_score < score:
            max_score = score
            max_k = k
        
    scores.append({'KNN':{'score':max_score, 'k':max_k}})
    
    parameters_range = [2,3,4,5,7,10,12,15]
    scores_clf = []
    max_score = max_d = max_m = 0
    for d in parameters_range:
        for m in parameters_range:
            clf = DecisionTreeClassifier(max_depth=d,min_samples_split=m)
            clf.fit(X_train,y_train)
            y_pred_DT = clf.predict(X_test)
            score = metrics.accuracy_score(y_test, y_pred_DT)
            if max_score < score:
                max_score = score
                max_d = d
                max_m = m
    scores.append({'DecisionTreeClassifier':{'score':str(max_score),'m':str(max_m), 'd':str(max_d)}})
    
    gnb = GaussianNB()
    gnb.fit(X_train, y_train) 
    y_pred_NB = gnb.predict(X_test) 
    score_train = metrics.accuracy_score(y_test, y_pred_NB)

    scores.append({'GaussianNB':{'score':score_train}})
    return scores
################################################

In [13]:
# Test followed df
testsResults = []
testFollowedX = follow_df[['Year','Coach1Wins','Coach1Lose','Total1','Precentage1','Coach2Wins','Coach2Lose','Total2','Precentage2']]
testFollowedY = follow_df['0_diff_Test1']
run_methods_estimators(testFollowedX,testFollowedY)

[{'KNN': {'score': 0.825, 'k': 11}},
 {'DecisionTreeClassifier': {'score': '0.85', 'm': '2', 'd': '5'}},
 {'GaussianNB': {'score': 0.75}}]

In [120]:
year_count = df.groupby(['Year'])['num_of_coaches'].count()
years = []
switches = []
for index, value in year_count.iteritems():
    years.append(index)
    switches.append(value)
fig = go.Figure(data=go.Scatter(x=years, y=switches))
fig.update_layout(title='NBA Coach switches by years',
                   xaxis_title='Years',
                   yaxis_title='Number of switches')
fig.show()

In [123]:
prec_count = df.groupby(['Precentage1'])['num_of_coaches'].count()
prec = []
switches = []
for index, value in prec_count.iteritems():
    prec.append(index)
    switches.append(value)
fig = go.Figure(data=go.Scatter(x=prec, y=switches))
fig.update_layout(title='NBA Coach switches by precentage of succsess',
                   xaxis_title='Precentage of succsess',
                   yaxis_title='Nuber of switches')
fig.show()

In [146]:
switch_succsess_0 = follow_df.groupby('0_diff_Test1')['Year'].count()
switch_succsess_5 = follow_df.groupby('5_diff_Test1')['Year'].count()
succ0=[]
succ5=[]
for indx, value in switch_succsess_0.iteritems():
    succ0.append(value)
succ0.reverse()
for indx, value in switch_succsess_5.iteritems():
    succ5.append(value)
succ5.reverse()
fig = go.Figure(data=[
    go.Bar(name='Difference larger then 0%', x=['Switch Succeeded', 'Switch Failed'], y=succ0),
    go.Bar(name='Difference larger then 5%', x=['Switch Succeeded', 'Switch Failed'], y=succ5)
])
fig.update_layout(title='Did the switch succeeded?', barmode='group')
fig.show()